### Step 1: Load Staging Data
We assume that the CSV files have already been loaded into the staging tables in SQL Server. Now, we connect to the database to preview the staging data.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

engine = create_engine(
    f"mssql+pyodbc://{os.getenv('SQL_USER')}:{os.getenv('SQL_PASSWORD')}@{os.getenv('SQL_SERVER')}:{os.getenv('SQL_PORT')}/{os.getenv('SQL_DB')}?driver=ODBC+Driver+17+for+SQL+Server"
)

query = '''
SELECT p.product_id, p.product_name, b.brand_name, c.category_name, p.model_year, p.list_price
FROM stg_products p
LEFT JOIN stg_brands b ON p.brand_id = b.brand_id
LEFT JOIN stg_categories c ON p.category_id = c.category_id
'''

df_products = pd.read_sql(query, engine)
df_products.head()


### Step 2: Understand the Goal
`dim_product` is a Type 1 dimension. We overwrite any changes without tracking history.
We need to compare the source with the existing dimension, and either update or insert.

In [ ]:
merge_sql = '''
MERGE dim_product AS target
USING (
    SELECT
        p.product_id,
        p.product_name,
        c.category_name,
        b.brand_name,
        p.model_year,
        p.list_price
    FROM stg_products p
    LEFT JOIN stg_brands b ON p.brand_id = b.brand_id
    LEFT JOIN stg_categories c ON p.category_id = c.category_id
) AS source
ON target.product_id = source.product_id
WHEN MATCHED THEN
    UPDATE SET
        product_name = source.product_name,
        category_name = source.category_name,
        brand_name = source.brand_name,
        model_year = source.model_year,
        list_price = source.list_price
WHEN NOT MATCHED BY TARGET THEN
    INSERT (product_id, product_name, category_name, brand_name, model_year, list_price)
    VALUES (source.product_id, source.product_name, source.category_name, source.brand_name, source.model_year, source.list_price);
'''

with engine.begin() as conn:
    conn.execute(merge_sql)